In [ ]:
!pip install --upgrade datasets huggingface_hub
import os
os.environ["HF_TOKEN"]= ""
!huggingface-cli whoami


brotherhao2605


In [ ]:
!pip install -q datasets python-docx

from datasets import load_dataset
from docx import Document
import pandas as pd
import re

dataset = load_dataset("TruongSinhAI/DEEPCAD-Text2Json-EnVi")




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

train_en-00000-of-00003.parquet:   0%|          | 0.00/82.8M [00:00<?, ?B/s]

train_en-00001-of-00003.parquet:   0%|          | 0.00/82.2M [00:00<?, ?B/s]

...

Generating val_vi split:   0%|          | 0/8814 [00:00<?, ? examples/s]

Generating test_en split:   0%|          | 0/7924 [00:00<?, ? examples/s]

Generating test_vi split:   0%|          | 0/7924 [00:00<?, ? examples/s]

Generating subset1000_en split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating subset1000_vi split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
df_vi = pd.DataFrame(dataset["train_vi"])

df_vi.head(1)

,uid,input,output,prompt
0,0035/00353613,**Phần 1: Lăng trụ chữ nhật ba chiều với đỉnh ...,"{""final_shape"": """", ""parts"": {""part_1"": {""coor...",\n<objective>\n Generate a JSON file describi...


In [ ]:
df_vi.describe()

In [ ]:
doc = Document('/content/xml_prompt.docx')
template = "\n".join([p.text for p in doc.paragraphs])

import re

def fill_template(desc: str) -> str:
    """
    Bỏ qua <description> đầu tiên, chèn desc vào giữa cặp <description> thứ hai...</description>.
    Nếu template không có đủ thẻ mở/đóng, sẽ báo lỗi rõ ràng.
    """
    open_tags = list(re.finditer(r'<description>', template))
    if len(open_tags) < 2:
        raise ValueError("Template không có 2 thẻ <description>")
    second_open = open_tags[1]

    close_tags = list(re.finditer(r'</description>', template))
    close_tag = next((m for m in close_tags if m.start() > second_open.end()), None)
    if close_tag is None:
        raise ValueError("Không tìm thấy </description> tương ứng sau tag thứ hai")

    start_idx = second_open.end()
    end_idx = close_tag.start()


    return (
        template[:start_idx]
        + "\n"
        + desc
        + "\n"
        + template[end_idx:]
    )

df_en['prompt'] = df_en['input'].apply(fill_template)
df_vi['prompt'] = df_vi['input'].apply(fill_template)

df_en[['input', 'prompt']].head(1)
df_vi[['input', 'prompt']].head(1)


In [ ]:
!pip install -q transformers
from tqdm import tqdm
from transformers import AutoTokenizer

# model_name = "meta-llama/Llama-3.1-8B-Instruct"
model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
from tqdm.auto import tqdm

texts_vi = df_vi['output'].tolist()

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
from tqdm.auto import tqdm
from transformers import AutoTokenizer

tqdm.pandas(desc="Tính token length")

def count_tokens(text):
    ids = tokenizer.encode(
        text,
        add_special_tokens=True,
        truncation=False,
        padding=False
    )
    return len(ids)


df_vi['token_len'] = df_vi['output'].progress_apply(count_tokens)



print("VI prompts token length:")
print(df_vi['token_len'].describe(), end='\n\n')



Tính token length:   0%|          | 0/155783 [00:00<?, ?it/s]

VI prompts token length:
count    155783.000000
mean        608.457226
std         448.012062
min         181.000000
25%         311.000000
50%         445.000000
75%         768.000000
max        3296.000000
Name: token_len, dtype: float64



In [ ]:
top10_vi = df_vi.sort_values(by='token_len', ascending=False).head(10)
print("Top 10 prompt VI dài nhất:")
print(top10_vi[["uid", 'token_len']])


Top 10 prompt VI dài nhất:
                  uid  token_len
1314    0077/00774557       3296
30487   0000/00000718       3133
64012   0062/00628692       3130
807     0035/00351884       3129
30700   0000/00001616       3124
111474  0072/00720019       3124
33441   0080/00803577       3073
60659   0090/00906422       3064
60838   0090/00900244       3064
149344  0084/00840422       3042


In [ ]:
top10_en = df_en.sort_values(by='token_len', ascending=False).head(10)
print("Top 10 prompt EN dài nhất:")
print(top10_en[["uid", 'token_len']])

In [ ]:
df_cmp = pd.DataFrame({
    'uid': df_en['uid'].reset_index(drop=True),
    'en_len': df_en['token_len'].reset_index(drop=True),
    'vi_len': df_vi['token_len'].reset_index(drop=True)
})
df_cmp['diff'] = df_cmp['vi_len'] - df_cmp['en_len']
print("Thống kê len diff (vi_len -en_len):")
print(df_cmp['diff'].describe(), end='\n\n')

In [ ]:
uid = "0077/00774557"

prompt_text = df_vi[df_vi['uid'] == uid]['prompt'].iloc[0]
print(count_tokens(prompt_text))

In [ ]:
df_cmp_sorted = df_cmp.sort_values(by='diff', ascending=True)
df_cmp_sorted.head(20)

In [ ]:
Q1 = df_cmp['diff'].quantile(0.25)
Q3 = df_cmp['diff'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = 0
upper_bound = Q3 + 1.5 * IQR
outliers = df_cmp[(df_cmp['diff'] <= lower_bound) | (df_cmp['diff'] > upper_bound)]

print(f"\nIQR = {IQR:.2f}")
print(f"Lower bound = {lower_bound:.2f}")
print(f"Upper bound = {upper_bound:.2f}")
print(f"\nSố lượng outliers phát hiện: {len(outliers)}")
display(outliers)

In [ ]:
check_id = "0005/0052367"
(df_en[df_en['uid']== check_id])


In [ ]:
check_id= "0077/00774557"
(df_vi[df_vi['uid']==check_id])

,uid,input,output,prompt,token_len
1314,0077/00774557,Hướng dẫn Phần 1 Xây dựng một hình lăng trụ ch...,"{""final_shape"": """", ""parts"": {""part_1"": {""coor...",\n<objective>\n Generate a JSON file describi...,3296


In [ ]:
n_positive = (df_cmp['diff'] > 0).sum()
n_negative = (df_cmp['diff'] < 0).sum()
print(f"Số diff > 0: {n_positive}")
print(f"Số diff < 0: {n_negative}")


In [ ]:
df_cmp.describe()

In [ ]:
outliers.describe()

In [ ]:
outliers.head()

In [ ]:
from transformers import AutoTokenizer

#tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", use_fast=True)

tokenizer_qwen = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct", use_fast=True)

def count_tokens(text: str, tokenizer) -> int:
    """
    Encode `text` with `tokenizer` và trả về số token.
    """
    tokens = tokenizer.encode(text, add_special_tokens=True)
    return len(tokens)

input_text = """
<objective> The objective of this prompt is to generate JSON sketch-and-extrusion steps to build a Computer Aid Design (CAD) model of an object. </objective> <instruction> You will be given a set of steps to create a Computer Aid Design (CAD) model in the <description> part. Follow the steps carefully and ensure that you understand each part of the process. The final output should be a JSON script that follows the <template> structure provided below. The JSON file should include all the necessary details about the CAD model, including sketch-and-extrusion steps. Make sure to validate the JSON file to ensure it is correctly formatted before submission. </instruction> <template> { "final_shape": "", "parts": { "<part_id>": { "coordinate_system": { "Euler Angles": [<number>, <number>, <number>], "Translation Vector": [<number>, <number>, <number>] }, "sketch": { "<face_id>": { "<loop_id>": { // one or more geometric primitives; repeat as needed // For a line: "<primitive_id>": { "Start Point": [<number>, <number>], "End Point": [<number>, <number>] } // For an arc: "<primitive_id>": { "Start Point": [<number>, <number>], "Mid Point": [<number>, <number>], "End Point": [<number>, <number>] } // For a circle: "<primitive_id>": { "Center": [<number>, <number>], "Radius": <number> } } // additional loops if any } // additional faces if any }, "extrusion": { "extrude_depth_towards_normal": <number>, "extrude_depth_opposite_normal": <number>, "sketch_scale": <number>, "operation": "<string>" }, "description": { "shape": "", "length": <number>, "width": <number>, "height": <number> } } // additional parts if any } } </template> <examples> <good_example> </good_example> <bad_example> A bad response would be one that does not follow the template structure, is missing or incorrect information about parts, faces, loops, or contains errors in the JSON formatting. </bad_example> </examples> <description> Hướng dẫn Mô hình CAD Xây dựng một vật thể hình trụ có mặt trên và mặt dưới phẳng, và một lỗ tròn ở trung tâm. Vật thể đối xứng và có bề mặt nhẵn. Phần 1: Tạo một hệ thống tọa độ mới với các thuộc tính sau: - Góc Euler: [0.0, 0.0, 0.0] - Vectơ Tịnh Tiến: [0.0, 0.0, 0.0225] Bắt đầu bằng cách tạo một mặt với vòng lặp và các đường sau: - Vòng lặp 1: - Đường 1: [0.0, 0.0] đến [0.615, 0.0] - Đường 2: [0.615, 0.0] đến [0.615, 0.75] - Đường 3: [0.615, 0.75] đến [0.0, 0.75] - Đường 4: [0.0, 0.75] đến [0.0, 0.0] Chia tỷ lệ bản phác thảo theo hệ số 0.75. Đùn bản phác thảo 0.0225 đơn vị theo hướng ngược lại của vectơ pháp tuyến. Phần 2: Tạo một hệ thống tọa độ mới với các thuộc tính sau: - Góc Euler: [0.0, 0.0, 0.0] - Vectơ Tịnh Tiến: [0.1275, 0.195, 0.0225] Bắt đầu bằng cách tạo các mặt sau với các vòng lặp và đường/cung tương ứng: Mặt 1: - Vòng lặp 1: - Cung 1: [0.0, 0.18] đến [0.0527, 0.0527] đến [0.18, 0.0] - Đường 1: [0.18, 0.0] đến [0.18, 0.0075] - Cung 2: [0.18, 0.0075] đến [0.058, 0.058] đến [0.0075, 0.18] - Đường 2: [0.0075, 0.18] đến [0.0, 0.18] Mặt 2: - Vòng lặp 1: - Đường 1: [0.0, 0.18] đến [0.0075, 0.18] - Cung 1: [0.0075, 0.18] đến [0.058, 0.302] đến [0.18, 0.3525] - Đường 2: [0.18, 0.3525] đến [0.18, 0.36] - Cung 2: [0.18, 0.36] đến [0.0527, 0.3073] đến [0.0, 0.18] Mặt 3: - Vòng lặp 1: - Cung 1: [0.18, 0.0] đến [0.3073, 0.0527] đến [0.36, 0.18] - Đường 1: [0.36, 0.18] đến [0.3525, 0.18] - Cung 2: [0.3525, 0.18] đến [0.302, 0.058] đến [0.18, 0.0075] - Đường 2: [0.18, 0.0075] đến [0.18, 0.0] Mặt 4: - Vòng lặp 1: - Cung 1: [0.18, 0.3525] đến [0.302, 0.302] đến [0.3525, 0.18] - Đường 1: [0.3525, 0.18] đến [0.36, 0.18] - Cung 2: [0.36, 0.18] đến [0.3073, 0.3073] đến [0.18, 0.36] - Đường 2: [0.18, 0.36] đến [0.18, 0.3525] Chia tỷ lệ bản phác thảo theo hệ số 0.36. Đùn bản phác thảo 0.15 đơn vị theo hướng vectơ pháp tuyến và 0.0 đơn vị theo hướng vectơ pháp tuyến ngược lại. Phần 3: Tạo một hệ thống tọa độ mới với các thuộc tính sau: - Góc Euler: [0.0, 0.0, 0.0] - Vectơ Tịnh Tiến: [0.0975, 0.165, 0.0225] Bắt đầu bằng cách tạo các mặt sau với các vòng lặp và hình tròn tương ứng: Mặt 1: - Vòng lặp 1: - Hình tròn 1: [0.21, 0.21], Bán kính: 0.21 Mặt 2: - Vòng lặp 1: - Hình tròn 1: [0.21, 0.21], Bán kính: 0.195 Chia tỷ lệ bản phác thảo theo hệ số 0.42. Đùn bản phác thảo 0 đơn vị theo hướng vectơ pháp tuyến và 0.0075 đơn vị theo hướng vectơ pháp tuyến ngược lại. Phần 4: Tạo một hệ thống tọa độ mới với các thuộc tính sau: - Góc Euler: [0.0, 0.0, 0.0] - Vectơ Tịnh Tiến: [0.0675, 0.135, 0.0225] Bắt đầu bằng cách tạo các mặt sau với các vòng lặp và hình tròn tương ứng: Mặt 1: - Vòng lặp 1: - Hình tròn 1: [0.24, 0.24], Bán kính: 0.24 Mặt 2: - Vòng lặp 1: - Hình tròn 1: [0.24, 0.24], Bán kính: 0.225 Chia tỷ lệ bản phác thảo theo hệ số 0.48. Đùn bản phác thảo 0 đơn vị theo hướng vectơ pháp tuyến và 0.0075 đơn vị theo hướng vectơ pháp tuyến ngược lại. Phần 5: Tạo một hệ thống tọa độ mới với các thuộc tính sau: - Góc Euler: [0.0, 0.0, 0.0] - Vectơ Tịnh Tiến: [0.0375, 0.105, 0.0225] Bắt đầu bằng cách tạo các mặt sau với các vòng lặp và hình tròn tương ứng: Mặt 1: - Vòng lặp 1: - Hình tròn 1: [0.27, 0.27], Bán kính: 0.27 Mặt 2: - Vòng lặp 1: - Hình tròn 1: [0.27, 0.27], Bán kính: 0.255 Chia tỷ lệ bản phác thảo theo hệ số 0.54. Đùn bản phác thảo 0 đơn vị theo hướng vectơ pháp tuyến và 0.0075 đơn vị theo hướng vectơ pháp tuyến ngược lại. Phần 6: Tạo một hệ thống tọa độ mới với các thuộc tính sau: - Góc Euler: [0.0, 0.0, 0.0] - Vectơ Tịnh Tiến: [0.0075, 0.075, 0.0225] Bắt đầu bằng cách tạo các mặt sau với các vòng lặp và hình tròn tương ứng: Mặt 1: - Vòng lặp 1: - Hình tròn 1: [0.3, 0.3], Bán kính: 0.3 Mặt 2: - Vòng lặp 1: - Hình tròn 1: [0.3, 0.3], Bán kính: 0.285 Chia tỷ lệ bản phác thảo theo hệ số </description>
"""

#n_llama = count_tokens(input_text, tokenizer_llama)
n_qwen  = count_tokens(input_text, tokenizer_qwen)

#print(f"Llama-3.1-8B-Instruct: {n_llama} token")
print(f"Qwen2.5-7B-Instruct:    {n_qwen} token")


In [ ]:
outliers[['uid']].to_csv('uid_outliers.csv', index=False)


In [ ]:
df_clean = df_cmp[~df_cmp['uid'].isin(outliers['uid'])]
df_clean.describe()